In [ ]:
# https://medium.com/@kevinossner/how-much-can-you-deadlift-96f84ca5f3ee

In [ ]:
# https://www.openpowerlifting.org./rankings/raw/fully-tested/men/2021

In [1]:
import numpy as np
import pandas as pd
import urllib.request as urllib
from bs4 import BeautifulSoup


import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette(sns.diverging_palette(220, 20, n=7))
%matplotlib inline
import time
import random

# df = pd.read_csv(fname, parse_dates=True)

In [2]:
def lbs_to_kg(lbs):
    return lbs * 0.453592

def kg_to_lbs(kg):
    return kg * 2.20462

In [3]:
def get_soup(url):
    """ Returns the BeautifulSoup object for website of the given category name in the 
        given CL webpage's homepage
    
    Args:
        url(String): the given URL
            
    Returns:
        soup(bs4.BeautifulSoup): the BeautifulSoup object representation of the desired category page 
    """
    
    # Open the target category page
    html = urllib.urlopen(url)
    
    # Create a BeautifulSoup object after the HTML page is read
    soup = BeautifulSoup(html.read())

    # Close the urllib connection to avoid issues with the website
    html.close()
    
    return soup

In [13]:
target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid=122663'
soup = get_soup(target_url)
choices = soup.find('optgroup', attrs={'label': 'USAPL Nationals 2022 - Male'})
options = choices.find_all('option')
url_class_map = {}
for option in options:
    w_class = ''
    # Handle 140+ class
    if option.get_text()[-1] == '+':
        w_class = 141.0
    else:
        w_class = float(option.get_text()) * -1
    if w_class >= 52:
        url = option['value']
        url_class_map[url] = w_class
print(url_class_map)
#     print(choice.attrs)
#     print(choice.get_text())

{'122643': 52.0, '122663': 56.0, '122644': 60.0, '122645': 67.5, '122646': 75.0, '122647': 82.5, '122648': 90.0, '122649': 100.0, '122650': 110.0, '122651': 125.0, '122673': 140.0, '122671': 141.0}


In [15]:
target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120362&categoryid=59&weightclassid=122653'
soup = get_soup(target_url)
choices = soup.find('optgroup', attrs={'label': 'USAPL Nationals 2022 - Female'})
options = choices.find_all('option')
url_class_map = {}
for option in options:
    w_class = ''
    # Handle 100+ class
    if option.get_text()[-1] == '+':
        w_class = 101.0
    else:
        w_class = float(option.get_text()) * -1
    if w_class >= 44:
        url = option['value']
        url_class_map[url] = w_class
print(url_class_map)

{'122653': 44.0, '122654': 48.0, '122655': 52.0, '122656': 56.0, '122657': 60.0, '122658': 67.5, '122659': 75.0, '122660': 82.5, '122661': 90.0, '122672': 100.0, '122670': 101.0}


In [16]:
target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120362&categoryid=59&weightclassid='

df2 = pd.DataFrame(columns = ['Name', 'Weight Class', 'Lift', 'Weight (kg)', 'Date'])

for url, w_class in url_class_map.items():
    page = f'{target_url}{url}'
    soup = get_soup(page)
    body = soup.find_all('tbody')[1]
    # print(body)
    # titles = body.find_all('th', {'colspan' : '9'})
    records = body.find_all('tr', class_=None)
    lift = ''
    for count, record in enumerate(records):
        # even tags are lift title
        if count > 0 and (count-1)%2 == 0:
            lift = record.get_text().strip()
    #         print(lift)
    #         print(record.get_text().strip())
        # odd tags are info
        if count > 0 and (count-1)%2 == 1:
            infos = record.find_all('td')
            name = infos[1]
            weight_class = infos[0]
            weight = infos[2]
            date = infos[3]
            # Create a temporary dictionary to store the information of the current post
            temp_dict = {'Name': name, 'Weight Class': w_class, 'Lift': lift, 'Weight (kg)': weight, 'Date': date}
            # Append the current post's information to the df DataFrame to create its respective row
            df2 = df2.append(temp_dict, ignore_index=True)
    #         for info in infos:
    #       except  print(info.get_text().strip())
    time.sleep(random.randint(2,3))

df2

,Name,Weight Class,Lift,Weight (kg),Date
0,[[Stephanie Rattunde]],44.0,Raw Open - Squat,[85 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
1,[[Stephanie Rattunde]],44.0,Raw Open - Bench press,[62.5 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
2,[[Stephanie Rattunde]],44.0,Raw Open - Bench press single lift,[62.5 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
3,[[Stephanie Rattunde]],44.0,Raw Open - Deadlift,[125 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
4,[[Stephanie Rattunde]],44.0,Raw Open - Deadlift single lift,[125 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
...,...,...,...,...,...
61,[[Mahailya Reeves]],101.0,Raw Open - Bench press,[167.5 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
62,[[Mahailya Reeves]],101.0,Raw Open - Bench press single lift,[167.5 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
63,[[Sarah Brenner]],101.0,Raw Open - Deadlift,[260 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
64,[[Sarah Brenner]],101.0,Raw Open - Deadlift single lift,[260 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...


In [18]:
# df2['Name'] = df2['Name'].apply(lambda x: x.get_text())
# # df['Weight Class'] = df['Weight Class'].apply(lambda x: float(x.get_text()) * -1)
# df2['Weight (kg)'] = df2['Weight (kg)'].apply(lambda x: float(x.get_text().split()[0]) if len(x.get_text())>0 else np.nan)
# df2['Date'] = df2['Date'].apply(lambda x: re.sub(r"[\n\t\s]*", "", x.get_text()))
df2['Sex'] = 'F'
df2

,Name,Weight Class,Lift,Weight (kg),Date,Sex
0,Stephanie Rattunde,44.0,Raw Open - Squat,85.0,10/15/2015,F
1,Stephanie Rattunde,44.0,Raw Open - Bench press,62.5,10/15/2015,F
2,Stephanie Rattunde,44.0,Raw Open - Bench press single lift,62.5,10/15/2015,F
3,Stephanie Rattunde,44.0,Raw Open - Deadlift,125.0,10/15/2015,F
4,Stephanie Rattunde,44.0,Raw Open - Deadlift single lift,125.0,10/15/2015,F
...,...,...,...,...,...,...
61,Mahailya Reeves,101.0,Raw Open - Bench press,167.5,10/16/2019,F
62,Mahailya Reeves,101.0,Raw Open - Bench press single lift,167.5,10/16/2019,F
63,Sarah Brenner,101.0,Raw Open - Deadlift,260.0,10/16/2019,F
64,Sarah Brenner,101.0,Raw Open - Deadlift single lift,260.0,10/16/2019,F


In [5]:
# Create the DataFrame and populate it with Data

# target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid=122643'
# target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid=122663'
target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid='

df = pd.DataFrame(columns = ['Name', 'Weight Class', 'Lift', 'Weight (kg)', 'Date'])

for url, w_class in url_class_map.items():
    page = f'{target_url}{url}'
    soup = get_soup(page)
    body = soup.find_all('tbody')[1]
    # print(body)
    # titles = body.find_all('th', {'colspan' : '9'})
    records = body.find_all('tr', class_=None)
    lift = ''
    for count, record in enumerate(records):
        # even tags are lift title
        if count > 0 and (count-1)%2 == 0:
            lift = record.get_text().strip()
    #         print(lift)
    #         print(record.get_text().strip())
        # odd tags are info
        if count > 0 and (count-1)%2 == 1:
            infos = record.find_all('td')
            name = infos[1]
            weight_class = infos[0]
            weight = infos[2]
            date = infos[3]
            # Create a temporary dictionary to store the information of the current post
            temp_dict = {'Name': name, 'Weight Class': w_class, 'Lift': lift, 'Weight (kg)': weight, 'Date': date}
            # Append the current post's information to the df DataFrame to create its respective row
            df = df.append(temp_dict, ignore_index=True)
    #         for info in infos:
    #       except  print(info.get_text().strip())
    time.sleep(random.randint(2,4))

df


,Name,Weight Class,Lift,Weight (kg),Date
0,[[Jasen Hinkel]],52.0,Raw Open - Squat,[165 kg],[\n\t\t\t \t\t\t\t07/01/2008\n\t\t\t \t\...
1,[[Caleb Tourres]],52.0,Raw Open - Bench press,[100 kg],[\n\t\t\t \t\t\t\t07/01/2008\n\t\t\t \t\...
2,[[Caleb Tourres]],52.0,Raw Open - Bench press single lift,[100 kg],[\n\t\t\t \t\t\t\t07/01/2008\n\t\t\t \t\...
3,[[Sean O'Leary]],52.0,Raw Open - Deadlift,[115 kg],[\n\t\t\t \t\t\t\t07/01/2008\n\t\t\t \t\...
4,[[ To be set]],52.0,Raw Open - Deadlift single lift,[],[\n]
...,...,...,...,...,...
67,[[Jaisyn Mike]],141.0,Raw Open - Bench press,[281 kg],[\n\t\t\t \t\t\t\t06/06/2018\n\t\t\t \t\...
68,[[Thomas Davis]],141.0,Raw Open - Bench press single lift,[300 kg],[\n\t\t\t \t\t\t\t05/18/2019\n\t\t\t \t\...
69,[[Jesus Olivares]],141.0,Raw Open - Deadlift,[400 kg],[\n\t\t\t \t\t\t\t12/12/2020\n\t\t\t \t\...
70,[[Jesus Olivares]],141.0,Raw Open - Deadlift single lift,[400 kg],[\n\t\t\t \t\t\t\t12/12/2020\n\t\t\t \t\...


In [6]:
import re
myString = re.sub(r"[\n\t\s]*", "", str(df['Date'][1].get_text()))
myString
# str(df['Date'][1]).strip()

'07/01/2008'

In [19]:
# df['Name'] = df['Name'].apply(lambda x: x.get_text())
# # df['Weight Class'] = df['Weight Class'].apply(lambda x: float(x.get_text()) * -1)
# df['Weight (kg)'] = df['Weight (kg)'].apply(lambda x: float(x.get_text().split()[0]) if len(x.get_text())>0 else np.nan)
# df['Date'] = df['Date'].apply(lambda x: re.sub(r"[\n\t\s]*", "", x.get_text()))
df['Sex'] = 'M'
df

,Name,Weight Class,Lift,Weight (kg),Date,Sex
0,Jasen Hinkel,52.0,Raw Open - Squat,165.0,07/01/2008,M
1,Caleb Tourres,52.0,Raw Open - Bench press,100.0,07/01/2008,M
2,Caleb Tourres,52.0,Raw Open - Bench press single lift,100.0,07/01/2008,M
3,Sean O'Leary,52.0,Raw Open - Deadlift,115.0,07/01/2008,M
4,To be set,52.0,Raw Open - Deadlift single lift,NaN,,M
...,...,...,...,...,...,...
67,Jaisyn Mike,141.0,Raw Open - Bench press,281.0,06/06/2018,M
68,Thomas Davis,141.0,Raw Open - Bench press single lift,300.0,05/18/2019,M
69,Jesus Olivares,141.0,Raw Open - Deadlift,400.0,12/12/2020,M
70,Jesus Olivares,141.0,Raw Open - Deadlift single lift,400.0,12/12/2020,M


In [11]:
df.groupby(['Lift', 'Weight Class']).first()
df.groupby(['Lift']).get_group('Raw Open - Deadlift')

,Name,Weight Class,Lift,Weight (kg),Date
3,Sean O'Leary,52.0,Raw Open - Deadlift,115.0,07/01/2008
9,Dalton Lacoe,56.0,Raw Open - Deadlift,232.5,10/15/2015
15,Charlie Yeng,60.0,Raw Open - Deadlift,275.0,11/14/2020
21,Daniel Clements,67.5,Raw Open - Deadlift,297.5,06/14/2021
27,Taylor Atwood,75.0,Raw Open - Deadlift,340.5,06/14/2021
33,Angelo Fortino,82.5,Raw Open - Deadlift,350.0,12/04/2021
39,Ian Bell,90.0,Raw Open - Deadlift,320.5,08/05/2012
45,Ashton Rouska,100.0,Raw Open - Deadlift,383.0,11/14/2020
51,David Woolson,110.0,Raw Open - Deadlift,373.0,12/04/2021
57,Michael Tuchscherer,125.0,Raw Open - Deadlift,382.5,10/20/2013


In [12]:
type(df['Date'][0])

str

In [23]:
total_data = df.append(df2, ignore_index=True)

In [24]:
# total_data.to_csv('current_usapl_american_raw_records.csv', index=False)